# Predict the bg+1:00 values

In [1]:
import os
import joblib
import numpy as np
import pandas as pd

In [2]:
# Load train and test data
train_data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'train.csv')
train_data_tmp = pd.read_csv(train_data_file, index_col=0, low_memory=False)
extra_train_data_file = os.path.join('..', '..', '..', '..', 'data', 'interim', 'all_test_2h.csv')
extra_train_data = pd.read_csv(extra_train_data_file, low_memory=False)

all_train_data = pd.concat([train_data_tmp, extra_train_data], axis=0)
all_train_data.head()

,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,bg-5:20,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [3]:
test_data_file = os.path.join('..', '..', '..', '..', 'data', 'raw', 'test.csv')
all_test_data = pd.read_csv(test_data_file, index_col=0, low_memory=False)

## Load patient-specific models

In [4]:
patient_ids = all_train_data['p_num'].unique()
patient_ids

array(['p01', 'p02', 'p03', 'p04', 'p05', 'p06', 'p10', 'p11', 'p12',
       'p15', 'p16', 'p18', 'p19', 'p21', 'p22', 'p24'], dtype=object)

# Prepare test results

In [5]:
submission = pd.DataFrame(index=all_test_data.index.copy())
submission['bg+1:00'] = -1.0

In [6]:
from pipelines import pipeline

patient_ids = all_train_data['p_num'].unique()
fitted_columns = []

specific_models = {}
for p_num in patient_ids:
    specific_model_file = f'LassoCVRegressor.{p_num}.model.pkl'
    print(f'Loading {specific_model_file}')
    model = joblib.load(specific_model_file)
    model_name = specific_model_file.replace('.model.pkl', '')

    print(f'Preparing data for {p_num}')
    train_data = all_train_data[all_train_data['p_num'] == p_num]
    train_data = pipeline.fit_transform(train_data)
    train_data = train_data.drop(columns=['p_num'])
    
    
    X = train_data.drop(columns=['bg+1:00'])
    y = np.log1p(train_data['bg+1:00'])

    print(f'Fitting {specific_model_file}')
    fitted_columns = X.columns
    model.fit(X=X, y=y)
    specific_models[p_num] = model

Loading LassoCVRegressor.p01.model.pkl
Preparing data for p01
Fitting LassoCVRegressor.p01.model.pkl


/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.10224468992851143, tolerance: 0.054191578573190353
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11345062382633841, tolerance: 0.054191578573190353
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increa

Loading LassoCVRegressor.p02.model.pkl
Preparing data for p02
Fitting LassoCVRegressor.p02.model.pkl
Loading LassoCVRegressor.p03.model.pkl
Preparing data for p03
Fitting LassoCVRegressor.p03.model.pkl
Loading LassoCVRegressor.p04.model.pkl
Preparing data for p04
Fitting LassoCVRegressor.p04.model.pkl
Loading LassoCVRegressor.p05.model.pkl
Preparing data for p05
Fitting LassoCVRegressor.p05.model.pkl


/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.859e+00, tolerance: 3.050e-01
  model = cd_fast.enet_coordinate_descent(


Loading LassoCVRegressor.p06.model.pkl
Preparing data for p06
Fitting LassoCVRegressor.p06.model.pkl


/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.020966003735622962, tolerance: 0.020480985089354253
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03850728392046676, tolerance: 0.020480985089354253
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ralf/Projects/learning-projects/datascience-bootcamp/sep24_bds_int_medical/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to incre

Loading LassoCVRegressor.p10.model.pkl
Preparing data for p10
Fitting LassoCVRegressor.p10.model.pkl
Loading LassoCVRegressor.p11.model.pkl
Preparing data for p11
Fitting LassoCVRegressor.p11.model.pkl
Loading LassoCVRegressor.p12.model.pkl
Preparing data for p12
Fitting LassoCVRegressor.p12.model.pkl
Loading LassoCVRegressor.p15.model.pkl
Preparing data for p15
Fitting LassoCVRegressor.p15.model.pkl
Loading LassoCVRegressor.p16.model.pkl
Preparing data for p16
Fitting LassoCVRegressor.p16.model.pkl
Loading LassoCVRegressor.p18.model.pkl
Preparing data for p18
Fitting LassoCVRegressor.p18.model.pkl
Loading LassoCVRegressor.p19.model.pkl
Preparing data for p19
Fitting LassoCVRegressor.p19.model.pkl
Loading LassoCVRegressor.p21.model.pkl
Preparing data for p21
Fitting LassoCVRegressor.p21.model.pkl
Loading LassoCVRegressor.p22.model.pkl
Preparing data for p22
Fitting LassoCVRegressor.p22.model.pkl
Loading LassoCVRegressor.p24.model.pkl
Preparing data for p24
Fitting LassoCVRegressor.p24.

In [7]:
from pipelines import pipeline

# Predict the bg+1:00 values
for p_num in patient_ids:
    print(f'----------------------------------------')
    print(f'Load model for {p_num}')
    model = specific_models[p_num]

    print(f'Prepare data for {p_num}')
    train_data = all_train_data[all_train_data['p_num'] == p_num]
    pipeline.fit_transform(train_data)
    
    test_data = all_test_data[all_test_data['p_num'] == p_num]
    if len(test_data) == 0:
        continue
    test_data = pipeline.transform(test_data)
    X = test_data.drop(columns=['p_num'])
    X = X[fitted_columns]

    print(f'Predict for {p_num}')
    prediction = np.expm1(model.predict(X=X))
    submission.loc[submission.index.isin(test_data.index), 'bg+1:00'] = prediction

----------------------------------------
Load model for p01
Prepare data for p01
Predict for p01
----------------------------------------
Load model for p02
Prepare data for p02
Predict for p02
----------------------------------------
Load model for p03
Prepare data for p03
----------------------------------------
Load model for p04
Prepare data for p04
Predict for p04
----------------------------------------
Load model for p05
Prepare data for p05
Predict for p05
----------------------------------------
Load model for p06
Prepare data for p06
Predict for p06
----------------------------------------
Load model for p10
Prepare data for p10
Predict for p10
----------------------------------------
Load model for p11
Prepare data for p11
Predict for p11
----------------------------------------
Load model for p12
Prepare data for p12
Predict for p12
----------------------------------------
Load model for p15
Prepare data for p15
Predict for p15
----------------------------------------
Load 

## Prepare the submission file

### Save the submission file

In [8]:
# check if there are missing predictions in the submission
missing_predictions = submission[submission['bg+1:00'] == -1.0]
missing_predictions

,bg+1:00
id,


In [9]:
submission.to_csv(f'submission-{os.path.basename(os.getcwd())}.csv')
submission

,bg+1:00
id,
p01_8459,9.120195
p01_8460,4.200169
p01_8461,8.111785
p01_8462,11.180762
p01_8463,7.992726
...,...
p24_256,6.461088
p24_257,9.187520
p24_258,7.030358
